In [ ]:
# Multiflux HPC Stress Test — Frozen Version
# CC BY-NC-SA 4.0 | Multiflux Research Lab

import os, time, gc
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import rfftn, irfftn, fftfreq, rfftfreq
from sklearn.cluster import MiniBatchKMeans

# =========================
# RESOLUTION CONFIGURATION
# =========================
N = 512            # <<< CHANGE ONLY THIS: 256 | 384 | 512
DS = 4
K_CLUSTERS = 12
SEED = 42
L = 2.0 * np.pi

OUTDIR = f"results/{N}"
os.makedirs(OUTDIR, exist_ok=True)

np.random.seed(SEED)

DTYPE = np.float32
CDTYPE = np.complex64

def log(msg):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

# =========================
# 1) Spectral grid
# =========================
log("Building spectral grid")
nr = N//2 + 1
kx = 2*np.pi * fftfreq(N, d=L/N).astype(DTYPE)
ky = 2*np.pi * fftfreq(N, d=L/N).astype(DTYPE)
kz = 2*np.pi * rfftfreq(N, d=L/N).astype(DTYPE)

KX = kx[:,None,None]
KY = ky[None,:,None]
KZ = kz[None,None,:]

K2 = KX**2 + KY**2 + KZ**2
K = np.sqrt(K2)
mask = K2 > 0

# =========================
# 2) Random velocity field
# =========================
log("Generating random spectral fields")

def rand_rfft():
    f = np.random.randn(N,N,N).astype(DTYPE)
    fh = rfftn(f).astype(CDTYPE)
    del f
    return fh

u_hat = rand_rfft()
v_hat = rand_rfft()
w_hat = rand_rfft()

scale = np.where(mask, K**(-5/6), 0).astype(CDTYPE)
u_hat *= scale; v_hat *= scale; w_hat *= scale

# Solenoidal projection
kdot = KX*u_hat + KY*v_hat + KZ*w_hat
u_hat -= KX * kdot / np.where(K2>0, K2, 1)
v_hat -= KY * kdot / np.where(K2>0, K2, 1)
w_hat -= KZ * kdot / np.where(K2>0, K2, 1)

# =========================
# 3) Physical velocity
# =========================
log("Inverse FFT")
u = irfftn(u_hat).astype(DTYPE)
v = irfftn(v_hat).astype(DTYPE)
w = irfftn(w_hat).astype(DTYPE)

KE = 0.5*np.mean(u*u+v*v+w*w)
log(f"Mean KE: {KE:.4e}")

# =========================
# 4) Gradients (spectral)
# =========================
def grad(f):
    fh = rfftn(f).astype(CDTYPE)
    fx = irfftn(1j*KX*fh).astype(DTYPE)
    fy = irfftn(1j*KY*fh).astype(DTYPE)
    fz = irfftn(1j*KZ*fh).astype(DTYPE)
    return fx,fy,fz

du = grad(u); dv = grad(v); dw = grad(w)

# =========================
# 5) Downsample + invariants
# =========================
s = slice(0,N,DS)
u,v,w = u[s,s,s], v[s,s,s], w[s,s,s]
du_dx,du_dy,du_dz = [x[s,s,s] for x in du]
dv_dx,dv_dy,dv_dz = [x[s,s,s] for x in dv]
dw_dx,dw_dy,dw_dz = [x[s,s,s] for x in dw]

omega_x = dw_dy - dv_dz
omega_y = du_dz - dw_dx
omega_z = dv_dx - du_dy

vort = np.sqrt(omega_x**2 + omega_y**2 + omega_z**2)
helicity = u*omega_x + v*omega_y + w*omega_z

S2 = du_dx**2 + dv_dy**2 + dw_dz**2
W2 = vort**2
Q = 0.5*(W2 - S2)

# =========================
# 6) Feature matrix
# =========================
X = np.column_stack([
    vort.ravel(),
    Q.ravel(),
    helicity.ravel()
])
X = (X - X.mean(0)) / (X.std(0)+1e-12)

# =========================
# 7) Clustering
# =========================
log("Clustering")
mbk = MiniBatchKMeans(n_clusters=K_CLUSTERS, random_state=SEED)
labels = mbk.fit_predict(X)

counts = np.bincount(labels)
volumes = counts / counts.sum()
N_eff = np.sum(volumes > 0.005)

log(f"N_eff = {N_eff}")

# =========================
# 8) Save results
# =========================
np.savetxt(f"{OUTDIR}/cluster_volumes.csv",
           np.column_stack([np.arange(K_CLUSTERS), volumes]),
           header="id,volume", delimiter=",")

# =========================
# 9) Visualization
# =========================
labels_ds = labels.reshape((N//DS,)*3)
mid = labels_ds.shape[0]//2

plt.figure(figsize=(6,5))
plt.imshow(labels_ds[:,:,mid], origin="lower")
plt.colorbar()
plt.title(f"Multiflux Slice N={N}")
plt.savefig(f"{OUTDIR}/multiflux_{N}.png", dpi=200)
plt.show()

log("Done.")


[20:09:36] Building spectral grid
[20:09:36] Generating random spectral fields


/tmp/ipykernel_30496/3437910205.py:62: RuntimeWarning: divide by zero encountered in power
  scale = np.where(mask, K**(-5/6), 0).astype(CDTYPE)


[20:09:55] Inverse FFT
[20:11:23] Mean KE: 1.4948e-04
